In [1]:
import pandas as pd
import numpy as np
import re
import nltk
import warnings
warnings.filterwarnings('ignore')
import emojis
import unicodedata
from bs4 import BeautifulSoup
from langdetect import detect, DetectorFactory
from nltk.stem.porter import PorterStemmer
from nltk.corpus import stopwords
from wordcloud import WordCloud, STOPWORDS
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from sklearn.feature_extraction.text import TfidfVectorizer
import matplotlib.pyplot as plt
from nltk.stem import WordNetLemmatizer
from nltk import pos_tag
from nltk.corpus import wordnet
nltk.download('wordnet')
from nltk.tokenize import regexp_tokenize, word_tokenize, RegexpTokenizer
from sklearn.utils import resample
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import SVC
from sklearn.ensemble import VotingClassifier
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix
# Libraries for deep learning
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, Flatten, Embedding, Dropout, Bidirectional, SpatialDropout1D
from tensorflow.keras.optimizers import Adam
from tensorflow.keras import regularizers
from tensorflow.keras.callbacks import ModelCheckpoint
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

/Users/tienhuynh/anaconda3/lib/python3.11/site-packages/pandas/core/arrays/masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/tienhuynh/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
2024-09-30 23:06:24.357322: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
review = pd.read_csv("./review_clean.csv")
review

,id,name,description,review,hotel_class,no_of_amenities,price_level,dist_to_metro,no_of_attractions,award,...,value,location,sleep_quality,rooms,check_in_front_desk,business_service,ratings,normalized_review,normalized_description,normalized_name
0,113317,Casablanca Hotel Times Square,Experience the magic of Times Square just step...,We had a whisper-quiet queen deluxe room couty...,4.0,50,very expensive,130.960,5,1,...,5.0,5.0,5.0,5.0,4.0,4.0,5.0,whisper quiet queen deluxe room coutyard room ...,experience magic time square step door escape ...,casablanca hotel time square
1,113317,Casablanca Hotel Times Square,Experience the magic of Times Square just step...,Great 3 night stay right after Thanksgiving. A...,4.0,50,very expensive,130.960,5,1,...,5.0,5.0,5.0,4.0,4.0,4.0,5.0,great night stay right thanksgiving agree prev...,experience magic time square step door escape ...,casablanca hotel time square
2,113317,Casablanca Hotel Times Square,Experience the magic of Times Square just step...,I hesitate to give this little gem a stellar r...,4.0,50,very expensive,130.960,5,1,...,5.0,5.0,5.0,5.0,4.0,4.0,5.0,hesitate give little gem stellar review fear a...,experience magic time square step door escape ...,casablanca hotel time square
3,113317,Casablanca Hotel Times Square,Experience the magic of Times Square just step...,You won't find a more conveniently located hot...,4.0,50,very expensive,130.960,5,1,...,5.0,5.0,5.0,4.0,4.0,4.0,5.0,find conveniently locate hotel nyc pace count ...,experience magic time square step door escape ...,casablanca hotel time square
4,113317,Casablanca Hotel Times Square,Experience the magic of Times Square just step...,This small boutique hotel right off Times Squa...,4.0,50,very expensive,130.960,5,1,...,5.0,5.0,4.0,4.0,4.0,4.0,5.0,small boutique hotel right time square perfect...,experience magic time square step door escape ...,casablanca hotel time square
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
239161,235513,Sofitel Washington DC,Sofitel Washington DC Lafayette Square represe...,Beautifully decorated and comfortable rooms ev...,4.0,79,very expensive,347.214,3,1,...,3.9,5.0,5.0,4.0,4.0,4.0,5.0,beautifully decorate comfortable room even tho...,sofitel washington dc lafayette square represe...,sofitel washington dc
239162,235513,Sofitel Washington DC,Sofitel Washington DC Lafayette Square represe...,I was there with my girlfriend for 4 days at t...,4.0,79,very expensive,347.214,3,1,...,3.9,5.0,5.0,4.0,4.0,4.0,5.0,girlfriend day end march location great close ...,sofitel washington dc lafayette square represe...,sofitel washington dc
239163,235513,Sofitel Washington DC,Sofitel Washington DC Lafayette Square represe...,Upscale hotel in downtown DC. Walking distance...,4.0,79,very expensive,347.214,3,1,...,3.9,5.0,5.0,4.0,4.0,4.0,4.0,upscale hotel downtown dc walk distance white ...,sofitel washington dc lafayette square represe...,sofitel washington dc
239164,235513,Sofitel Washington DC,Sofitel Washington DC Lafayette Square represe...,All expectations for service and accommodation...,4.0,79,very expensive,347.214,3,1,...,3.9,5.0,5.0,4.0,4.0,4.0,5.0,expectation service accommodation exceed probl...,sofitel washington dc lafayette square represe...,sofitel washington dc


In [3]:
review.isnull().sum()

id                        0
name                      0
description               0
review                    0
hotel_class               0
no_of_amenities           0
price_level               0
dist_to_metro             0
no_of_attractions         0
award                     0
no_of_reviews             0
service                   0
cleanliness               0
value                     0
location                  0
sleep_quality             0
rooms                     0
check_in_front_desk       0
business_service          0
ratings                   0
normalized_review         0
normalized_description    5
normalized_name           0
dtype: int64

In [4]:
review['ratings'].value_counts()

ratings
5.0    101589
4.0     76777
3.0     31063
2.0     15680
1.0     14057
Name: count, dtype: int64

In [5]:
def categorize_rating(rating):
    if rating <= 3:
        return 0
    else:
        return 1

# Apply the categorization function to the ratings column
review['target'] = review['ratings'].apply(categorize_rating)

In [6]:
review['target'].value_counts(normalize=True)

target
1    0.745783
0    0.254217
Name: proportion, dtype: float64

In [7]:

# # Separate majority and minority classes
# majority_class = review[review['target'] == 1]  # Assuming 1 is the majority class
# minority_class = review[review['target'] == 0]  # Assuming 0 is the minority class

# # Apply upsampling or downsampling
# # Example: Upsampling
# minority_upsampled = resample(minority_class, 
#                               replace=True,     # sample with replacement
#                               n_samples=len(majority_class),  # match to majority class
#                               random_state=42)

# # Combine majority class with upsampled minority class
# review_upsampled = pd.concat([majority_class, minority_upsampled])



In [8]:
# review_upsampled['target'].value_counts(normalize=True)

In [9]:
# #Countvectorizer
# # Use count vectorizer to check how many unique words there are
# cvec = CountVectorizer(stop_words='english') 
# cvec_df = pd.DataFrame(cvec.fit_transform(X_train_balanced['normalized_review']).todense(), columns=cvec.get_feature_names_out())
# cvec_df.shape

In [10]:
# def cmat(actual_y, predictions, dataset):
#     # Check the unique classes in the actual values and predictions
#     unique_classes = np.unique(np.concatenate((actual_y, predictions)))
#     num_classes = len(unique_classes)
    
#     # Create a classification report
#     print('Classification report for', dataset)
#     print(classification_report(actual_y, predictions))
#     print('')
    
#     # Create a confusion matrix
#     cm = confusion_matrix(actual_y, predictions)
    
#     # Dynamically create labels for the confusion matrix
#     labels = [f'Class {cls}' for cls in unique_classes]
    
#     # Convert confusion matrix to DataFrame
#     cm_df = pd.DataFrame(cm, columns=[f'Predicted {label}' for label in labels], 
#                          index=[f'Actual {label}' for label in labels])
    
#     print('Confusion matrix for', dataset)
#     print(cm_df)


In [11]:
# # Create a pipeline with Count Vectorizer and Logistic Regression
# pipe_cvec_lr = Pipeline([
#     ('cvec', CountVectorizer(stop_words='english')), 
#     ('lr', LogisticRegression(random_state=42))
# ])

# # Search over the following values of hyperparameters:
# pipe_cvec_lr_params = {
#     'cvec__max_features': [300], #100,200
#     'cvec__min_df': [0.001, 0.005], 
#     'cvec__max_df': [.9, .95], 
#     'cvec__ngram_range':[(1,1),(1,2)],  
#     'lr__penalty': ['l2'],
#     'lr__C': [.01,.1]
# }

# # Instantiate GridSearchCV
# gs_cvec_lr = GridSearchCV(pipe_cvec_lr, # Objects to optimise
#                           param_grid = pipe_cvec_lr_params, # Hyperparameters for tuning
#                           cv=10) # 10-fold cross validation

# # Fit model on to training data
# gs_cvec_lr.fit(X_train_balanced.values.ravel(), y_train_balanced)

# # Generate predictions on validation set
# cvec_lr_pred = gs_cvec_lr.predict(X_test)

# # Print best parameters
# print('Best parameters: ', gs_cvec_lr.best_params_)

# # Print accuracy scores
# print('Best CV score: ', gs_cvec_lr.best_score_)
# print('Training score:', gs_cvec_lr.score(X_train_balanced.values.ravel(), y_train_balanced))
# print('Validation score:', gs_cvec_lr.score(X_test, y_test))
# print('')

# # Print classification report and confusion matrix
# cmat(y_test, cvec_lr_pred, 'validation set')

In [12]:
# # Create a pipeline with Count Vectorizer and Naive Bayes
# pipe_cvec_nb = Pipeline([
#     ('cvec', CountVectorizer(stop_words='english')),
#     ('nb', MultinomialNB())
# ])

# # Search over the following values of hyperparameters:
# pipe_cvec_nb_params = {
#     'cvec__max_features': [500], #200
#     'cvec__min_df': [0.001, 0.005],
#     'cvec__max_df': [.9,.95], 
#     'cvec__ngram_range':[(1,1),(1,2)],  
# }

# # Instantiate GridSearchCV
# gs_cvec_nb = GridSearchCV(pipe_cvec_nb, # Objects to optimise
#                           param_grid = pipe_cvec_nb_params, # Hyperparameters for tuning
#                           cv=10) # 10-fold cross validation

# # Fit model on to training data
# gs_cvec_nb.fit(X_train_balanced.values.ravel(), y_train_balanced)

# # Generate predictions on validation set
# cvec_nb_pred = gs_cvec_nb.predict(X_test)

# # Print best parameters
# print('Best parameters: ', gs_cvec_nb.best_params_)

# # Print accuracy scores
# print('Best CV score: ', gs_cvec_nb.best_score_)
# print('Training score:', gs_cvec_nb.score(X_train_balanced.values.ravel(), y_train_balanced))
# print('Validation score:', gs_cvec_nb.score(X_test, y_test))
# print('')

# # Print classification report and confusion matrix
# cmat(y_test, cvec_nb_pred, 'validation set')

In [13]:
# # Create a pipeline with TF-IDF and Logistic Regression
# pipe_tvec_lr = Pipeline([
#     ('tvec', TfidfVectorizer(stop_words='english')),
#     ('lr', LogisticRegression(random_state=42))
# ])

# # Search over the following values of hyperparameters:
# pipe_tvec_lr_params = {
#     'tvec__max_features': [300], #100,200
#     'tvec__min_df': [0.001, 0.005], #2,3 
#     'tvec__max_df': [.9,.95], 
#     'tvec__ngram_range':[(1,1),(1,2)],  
#     'lr__penalty': ['l2'],
#     'lr__C': [.1, 1] #.1, .01
# }

# # Instantiate GridSearchCV
# gs_tvec_lr = GridSearchCV(pipe_tvec_lr, # Objects to optimise
#                           param_grid = pipe_tvec_lr_params, # Hyperparameters for tuning
#                           cv=10) # 10-fold cross validation

# # Fit model on to training data
# gs_tvec_lr.fit(X_train_balanced.values.ravel(), y_train_balanced)

# # Generate predictions on validation set
# tvec_lr_pred = gs_tvec_lr.predict(X_test)

# # Print best parameters
# print('Best parameters: ', gs_tvec_lr.best_params_)

# # Print accuracy scores
# print('Best CV score: ', gs_tvec_lr.best_score_)
# print('Training score:', gs_tvec_lr.score(X_train_balanced.values.ravel(), y_train_balanced))
# print('Validation score:', gs_tvec_lr.score(X_test, y_test))
# print('')

# # Print classification report and confusion matrix
# cmat(y_test, tvec_lr_pred, 'validation set')

In [14]:
# # Create a pipeline with TF-IDF and Naive Bayes
# pipe_tvec_nb = Pipeline([
#     ('tvec', TfidfVectorizer(stop_words='english')),
#     ('nb', MultinomialNB())
# ])

# # Search over the following values of hyperparameters:
# pipe_tvec_nb_params = {
#     'tvec__max_features': [500], #200
#     'tvec__min_df': [0.001, 0.005], #
#     'tvec__max_df': [.9,.95], 
#     'tvec__ngram_range':[(1,1),(1,2)],  
# }

# # Instantiate GridSearchCV
# gs_tvec_nb = GridSearchCV(pipe_tvec_nb, # Objects to optimise
#                           param_grid = pipe_tvec_nb_params, # Hyperparameters for tuning
#                           cv=10) # 10-fold cross validation

# # Fit model on to training data
# gs_tvec_nb.fit(X_train_balanced.values.ravel(), y_train_balanced)

# # Generate predictions on validation set
# tvec_nb_pred = gs_tvec_nb.predict(X_test)

# # Print best parameters
# print('Best parameters: ', gs_tvec_nb.best_params_)

# # Print accuracy scores
# print('Best CV score: ', gs_tvec_nb.best_score_)
# print('Training score:', gs_tvec_nb.score(X_train_balanced.values.ravel(), y_train_balanced))
# print('Validation score:', gs_tvec_nb.score(X_test, y_test))
# print('')

# # Print classification report and confusion matrix
# cmat(y_test, tvec_nb_pred, 'validation set')

In [15]:
review['normalized_description'].isnull().sum()

5

In [16]:
# Instantiate the sentiment intensity analyzer
sia = SentimentIntensityAnalyzer()

In [17]:
# Create an empty list to append the polarity scores
sia_list = []

# Loop through the text in the training dataset and create a dictionary of the VADER scores
for text in review['normalized_review']:
    sia_dict = {}
    sia_dict['neg'] = sia.polarity_scores(text)['neg'] # Negative score
    sia_dict['neu'] = sia.polarity_scores(text)['neu'] # Neutral score
    sia_dict['pos'] = sia.polarity_scores(text)['pos'] # Positive scores
    sia_dict['compound'] = sia.polarity_scores(text)['compound'] # Compound scores
    sia_list.append(sia_dict) # Append the dictionary of scores to the sia_list

In [18]:
# Create a dataframe from the sia_list
sia_df = pd.DataFrame(sia_list)

In [19]:
sia_df

,neg,neu,pos,compound
0,0.000,0.674,0.326,0.9468
1,0.017,0.688,0.295,0.9705
2,0.151,0.627,0.222,0.3612
3,0.117,0.604,0.279,0.8957
4,0.000,0.709,0.291,0.9826
...,...,...,...,...
239161,0.000,0.554,0.446,0.9750
239162,0.051,0.553,0.395,0.9413
239163,0.000,0.617,0.383,0.9468
239164,0.089,0.757,0.154,0.2472


In [20]:
def extract_score(compound):
    sentiment = ''
    
    if compound >= 0.05:
        sentiment = "positive"
    elif compound <= -0.05:
        sentiment = "negative"
    else:
        sentiment = "neutral"

    return sentiment


In [21]:
review['sentiment'] = sia_df['compound'].apply(extract_score)

In [22]:
review[review['normalized_description'].isnull()]

,id,name,description,review,hotel_class,no_of_amenities,price_level,dist_to_metro,no_of_attractions,award,...,sleep_quality,rooms,check_in_front_desk,business_service,ratings,normalized_review,normalized_description,normalized_name,target,sentiment
12111,98937,Astro Inn,.,"For spring break, we visited Houston and we ne...",0.0,3,low,0.0,0,0,...,5.0,1.0,4.0,4.0,1.0,spring break visit houston need place stay cou...,NaN,astro inn,0,negative
12112,98937,Astro Inn,.,You get what you pay for. This place was cheap...,0.0,3,low,0.0,0,0,...,1.0,1.0,4.0,4.0,1.0,get pay place cheap really need room since arr...,NaN,astro inn,0,negative
12113,98937,Astro Inn,.,we were told that the astro inn in houston tex...,0.0,3,low,0.0,0,0,...,5.0,4.0,4.0,4.0,1.0,tell astro inn houston texas really nice hotel...,NaN,astro inn,0,positive
77004,676721,River Park Motel & Suites,\n,This is a ghetto motel..they have hardcore por...,0.0,4,Not Specified,0.0,3,0,...,1.0,2.0,4.0,4.0,2.0,ghetto motel hardcore porno movie stream tv go...,NaN,river park motel suite,0,neutral
77005,676721,River Park Motel & Suites,\n,My husband and I decided to drive into Chicago...,0.0,4,Not Specified,0.0,3,0,...,5.0,5.0,5.0,4.0,4.0,husband decide drive chicago wisconsin weekend...,NaN,river park motel suite,1,positive


In [23]:
review['sentiment'].value_counts()

sentiment
positive    225716
negative     12059
neutral       1391
Name: count, dtype: int64

In [25]:
review.isnull().sum()

id                        0
name                      0
description               0
review                    0
hotel_class               0
no_of_amenities           0
price_level               0
dist_to_metro             0
no_of_attractions         0
award                     0
no_of_reviews             0
service                   0
cleanliness               0
value                     0
location                  0
sleep_quality             0
rooms                     0
check_in_front_desk       0
business_service          0
ratings                   0
normalized_review         0
normalized_description    5
normalized_name           0
target                    0
sentiment                 0
dtype: int64

In [26]:
review.to_csv("review_normalized.csv", index=False)